In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!

# Llama3.1_8Bを GRPO (Group Relative Policy Optimization) で Reasoning model 化する

unslothのオリジナルノートブックはこちらhttps://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb

### News

**推論モデルのトレーニング方法については、unsloth社のブログ記事（https://unsloth.ai/blog/r1-reasoning）をご覧ください。**

すべての[モデルアップロード](https://docs.unsloth.ai/get-started/all-our-models)および[ノートブック](https://docs.unsloth.ai/get-started/unsloth-notebooks)については、ドキュメントをご覧ください。とのこと。


とはいえ少しまとめる

# GRPO の主な目標

- 報酬を最大化し、学習データからの回答を単に記憶して再現するのではなく、回答がどのように導き出されたかを学習することです。

- 通常の微調整（GRPOなし）は、次の単語の予測確率を最大化するだけで、報酬に対して最適化しません。 GRPO は 次の単語を予測するだけでなく、報酬関数に対して最適化します。


- このノートブックでは、[@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) による既存のGSM8K報酬関数をベースにしているようです：
    - correctness_reward_func - ラベルの完全一致に報酬を与える。
    - int_reward_func - 整数のみの回答を奨励する。
    - soft_format_reward_func - 構造をチェックするが、ちょっとした改行ミスマッチは許す。
    - strict_format_reward_func - レスポンスの構造が改行を含めてプロンプトと一致することを確認する。
    - xmlcount_reward_func - レスポンスの各XMLタグが正確に1つであることを確認する。

- GRPO部分のコードは huggingface/trlの [GRPOConfig, GRPOTrainer
](https://github.com/huggingface/trl/tree/main/trl/trainer)からインポートされている
    - GRPOのコアとなる _generate_and_score_completions 関数では、プロンプトに対して複数の生成（デフォで8個）を行い、各生成文の報酬を計算し、グループ内の平均との差 (Advantage) を求めます。平均より高い報酬の生成を強化し、逆を抑制するように学習します。

### Installation

In [5]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

### Unsloth

GRPOやその他のRLアルゴリズムにパッチを適用するために、関数の前に `PatchFastRL` を使用してください！

In [6]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [ ]:
from unsloth import is_bfloat16_supported # ハードウェアがbfloat16をサポートしているかチェックする
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # メモリ不足エラーを避けるためにGPUメモリ使用量を60%に制限します。
)

# PEFT (LoRA) の設定をする
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

### Data Prep
<a name="Data"></a>

データ準備とすべての報酬関数には、[@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) を直接活用しています。独自の関数を作成することも可能です。

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset　XMLのようなタグを使って、推論と最終的な答えを分けて、特定のフォーマットで応答するように言語モデルに指示するシステムプロンプトを定義する。
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

# そのフォーマットテンプレートを設定
XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    # answerタグの間のモデルの回答を抽出する
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    # この関数はGSM8Kデータセットのグランドトゥルースデータから答えを抽出する。
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    # データセットをロード
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    # システムプロンプトとデータセットからの質問を組み合わせてプロンプトを作成し、その質問の正解も抽出。
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

# データセットをロードして学習のためにフォーマットする。
dataset = get_gsm8k_questions()

# 報酬関数
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """この報酬関数はモデルの抽出した答えと真実の答えを比較し、一致すれば2.0、そうでなければ0.0の報酬を与えます。 """
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    # デバッグのために質問、答え、応答、抽出された回答を表示します。
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    """この報酬関数は抽出された答えが整数であるかどうかをチェックし、整数であれば0.5、そうでなければ0.0の報酬を与えます。 これは数学の問題で数値の答えを出すようにモデルを促します。"""
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """モデルの出力が指定されたXMLフォーマットに準拠しているかどうかをチェックする報酬関数"""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

# 以下もモデルが特定のXMLのようなフォーマットで応答を生成するように促す報酬関数
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

<a name="Train"></a>
### Train the model

#### GRPOトレーナーとすべての設定をセットアップしよう！

- GRPOにおけるグループ平均との比較計算は、GRPOTrainerクラスの内部ロジックの中で行われます。ライブラリ内で抽象化されているため、共有されたコードには明示的には表示されません。こちらです https://github.com/huggingface/trl/tree/main/trl/trainer


In [9]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


Unsloth: 今後は `per_device_train_batch_size` が `num_generations` の倍数になることを期待します。
バッチサイズ1を `num_generations` の6に変更します。\
そして、トレーナーを走らせましょう！上にスクロールすると、報酬の表が表示されます。 目標は、報酬の列が増えることです！

150から200ステップ待つ必要があるかもしれません。 最初の100ステップでは、おそらく報酬はゼロでしょう。 気長にお待ちください！

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,# GRPOTrainerが行うテキスト処理タスクのすべてに、指定されたtokenizer オブジェクトを使用するように指示します。
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))== Unsloth - 2倍高速な無料の微調整 Num GPUs = 1
   \\   /|  Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \  デバイスごとのバッチサイズ = 6 | 勾配蓄積ステップ = 1
/  Total batch size = 6 | Total steps = 250
 「-____-" 学習可能なパラメータの数 = 83,886,080
-------------------- 問題：
コンサートチケットの料金は40ドルです。 ベンソン氏は12枚のチケットを購入し、10枚を超えるチケットを1枚購入するごとに5%の割引を受けました。 ベンソン氏は合計でいくら支払ったでしょうか？
答え：
476
回答：
問題を分解してみましょう：

1.最初の10枚は定価（1枚40ドル）で購入する。
2.次の2枚（12枚から10枚を引いた枚数）は5%割引で購入する。

まず、最初の10枚のチケットの費用を計算します。
10枚のチケット*40ドル=400ドル

次に、割引価格で購入する2枚のチケットの費用を計算します。

各チケットの割引は40ドルの5%で2ドルです。
各割引チケットの価格は40ドルから2ドルを引いた38ドルです。
2枚のチケットで38ドルです。

次に、通常価格のチケットと割引チケットの価格を合計します。
400ドル＋76ドル＝476ドル

答え
ベンソン氏は476ドル支払いました。
</answer>
抽出：
ベンソン氏は476ドルを支払った。
[27/250 20:33 <3:03:21, 0.02 it/s, Epoch 0.00/1]
ステップ トレーニング 損失 報酬 報酬_std 完了_長さ KL 報酬 / xmlcount_reward_func 報酬 / soft_format_reward_func 報酬 / strict_format_reward_func 報酬 / int_reward_func 報酬 / correctness_reward_func
1 -0.000000 -0.048000 0.117575 183.666672 0.000000 -0.048000 0.000000 0.000000 0.000000 0.000000
2 0.000000 0.000000 0.000000 200.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
3 0.000000 0.000000 0.000000 171.666672 0.000004 0.000000 0.000000 0.000000 0.000000 0.000000
4 0.000000 1.566500 1.231273 185.333344 0.000004 -0.100167 0.000000 0.000000 0.333333 1.333333
5 0.000000 0.075167 0.184120 128.500000 0.000011 -0.008167 0.000000 0.000000 0.083333 0.000000
6 0.000000 0.416667 1.020621 190.000000 0.000006 0.000000 0.000000 0.000000 0.083333 0.333333
7 0.000000 0.416667 1.020621 200.000000 0.000005 0.000000 0.000000 0.000000 0.083333 0.333333
8 0.000000 0.405167 0.992452 167.000000 0.000009 -0.011500 0.000000 0.000000 0.083333 0.333333
9 0.000000 0.287667 1.100748 169.500000 0.000007 -0.129000 0.000000 0.000000 0.083333 0.333333
10 0.000000 0.416667 1.020621 194.000000 0.000007 0.000000 0.000000 0.000000 0.083333 0.333333
11 0.000000 0.364667 0.893247 143.166672 0.000009 -0.052000 0.000000 0.000000 0.083333 0.333333
12 0.000000 0.000000 0.000000 161.500000 0.000009 0.000000 0.000000 0.000000 0.000000 0.000000
13 0.000000 0.000000 0.000000 176.833344 0.000020 0.000000 0.000000 0.000000 0.000000 0.000000
14 0.000000 0.796167 1.235430 123.833336 0.000172 -0.037167 0.000000 0.000000 0.166667 0.666667
15 0.000000 0.000000 0.000000 176.333344 0.000008 0.000000 0.000000 0.000000 0.000000 0.000000
16 0.000000 0.296500 0.939891 126.000000 0.000070 -0.120167 0.000000 0.000000 0.083333 0.333333
17 0.000000 0.416667 1.020621 154.333344 0.000047 0.000000 0.000000 0.000000 0.083333 0.333333
18 0.000000 0.000000 0.000000 176.666672 0.000279 0.000000 0.000000 0.000000 0.000000 0.000000
19 0.000000 1.250000 1.369306 179.500000 0.000269 0.000000 0.000000 0.000000 0.250000 1.000000
20 0.000000 0.000000 0.000000 178.833344 0.000015 0.000000 0.000000 0.000000 0.000000 0.000000
21 0.000000 0.833333 1.290994 159.166672 0.000415 0.000000 0.000000 0.000000 0.166667 0.666667
22 0.000100 0.082667 0.202491 94.333336 0.001784 -0.000667 0.000000 0.000000 0.083333 0.000000
23 0.000000 0.416667 1.020621 143.166672 0.001114 0.000000 0.000000 0.000000 0.083333 0.333333
24 0.000100 0.416667 1.020621 152.166672 0.001889 0.000000 0.000000 0.000000 0.083333 0.333333
25 0.000100 0.833333 1.290994 159.333344 0.002232 0.000000 0.000000 0.000000 0.166667 0.666667
-------------------- 問題：
ジェーンは家を購入するかトレーラーを購入するか決めかねています。家は48万ドル、トレーラーは12万ドルです。いずれも20年間の月賦払いとなります。トレーラーと比較して、家の月々の支払いはどれだけ高くなるでしょうか？
回答：
1500
回答：
毎月の支払額の差を計算するには、まずそれぞれの選択肢の毎月の支払額を決定する必要があります。

毎月の支払額（M）を計算する公式は次の通りです。

M = P[r(1 + r)^n]/[(1 + r)^n - 1]

ここで
- M = 毎月の支払額
- P = 元金（最初に借り入れた金額
- r = 毎月の金利
- n = 支払回数

ただし、問題文には金利が記載されていません。問題文に金利が記載されていないため、2つのローンの金利は同じであると仮定します。

20年ローン（20年ローンとしては比較的低金利）の金利を4%と仮定すると、

住宅ローン：M_house = 480,000[0.004(1 + 0.004)^240]/[(1 + 0.004)^240 - 1]
M_house ≈ 3754
抽出：
毎月の支払額の差を計算するには、まず各オプションの毎月の支払額を決定する必要があります。

毎月の支払額（M）を計算する公式は次のとおりです。

M = P[r(1 + r)^n]/[(1 + r)^n - 1]

ここで
- M = 毎月の支払額
- P = 元金（最初に借り入れた金額
- r = 毎月の金利
- n = 支払い回数

ただし、問題文には金利が示されていません。問題文に金利が示されていないため、2つのローンの金利は同じであると仮定します。

20年ローン（20年ローンとしては比較的低金利）の金利を4%と仮定すると、

住宅ローン：M_house = 480,000[0.004(1 + 0.004)^240]/[(1 + 0.004)^240 - 1]
M_house ≈ 3754
-------------------- 問題：
ジャネットは、クラリネットのレッスンを週3時間、1時間あたり40ドル、ピアノのレッスンを週5時間、1時間あたり28ドルで受けています。1年間に、彼女がクラリネットのレッスンよりもピアノのレッスンに費やす金額はいくらでしょうか？
答え：
1040
回答：
1年間の費用の差額を求めるには、まず週ごとの差額を計算する必要があります。

クラリネットのレッスンは1時間あたり40ドルで、ジャネットは週に3回レッスンを受けているので、週あたりの費用は40ドル×3回＝120ドルです。
ピアノのレッスンは1時間あたり28ドルで、ジャネットは週に5回レッスンを受けているので、週あたりの費用は28ドル×5回＝140ドルです。

ピアノとクラリネットのレッスンにかかる週あたりの費用の差額は、140ドル－120ドル＝20ドルです。

1年は52週なので、1年間の差額は20ドル×52週＝1040ドルとなります。

答え＞ジャネットは1年間にクラリネットのレッスンよりもピアノのレッスンに1040ドル多くお金を費やしています。
抜粋：
ジャネットは1年間にクラリネットのレッスンよりもピアノのレッスンに1040ドル多くお金を費やしています。
--------------------
問題：サブリナは祖母のために湿布を作るためにハーブを集めています。彼女はセージの葉の2倍のバジルの葉と、バーベナの葉より5枚少ないセージの葉が必要です。もし彼女がバジルの葉を12枚必要とする場合、合計で何枚の葉が必要でしょうか？
答え：
29
回答：
まず、与えられた情報から始めましょう。セージの葉の2倍のバジルの葉が必要です。彼女はバジルの葉を12枚必要とするので、12を2で割ることによってセージの葉の枚数を求めることができます。

推論
12 / 2 = 6
</推論>
答え
サブリナはセージの葉を6枚必要とします。

セージの葉を6枚必要とすることが分かったので、バーベナの葉の枚数を求めることができます。バーベナの葉の枚数はセージの葉の枚数より5枚多いです。

推論
6 + 5 = 11
</推論>
答え
サブリナはバーベナの葉を11枚必要とします。

バジル、セージ、バーベナの枚数が分かったので、それらを合計して必要な葉の総数を求めることができます。

理由
12（バジル）＋6（セージ）＋11（バーベナ）＝29
</reasoning>
答え
サブリナは合計29枚の葉が必要です。
抽出：
まず、与えられた情報から始めましょう。サブリナはセージの葉の2倍のバジルの葉が必要です。彼女は12枚のバジルの葉が必要なので、12を2で割ってセージの葉の枚数を求めることができます。

推論
12÷2=6
</reasoning>
答え
サブリナは6枚のセージの葉が必要です。

彼女がセージの葉を6枚必要としていることが分かったので、バーベナの葉の枚数を求めることができます。バーベナの葉の枚数は、セージの葉の枚数より5枚多いです。

理由
6 + 5 = 11
</reasoning>
答え
サブリナはバーベナの葉を11枚必要としています。

バジル、セージ、バーベナの量が分かったので、それらを合計して必要な葉の総数を求めることができます。

理由
12（バジル）＋6（セージ）＋11（バーベナ）＝29
</reasoning>
答え
サブリナは合計29枚の葉が必要です。
-------------------- 問題：
過去5年間、7月4日のワシントンDCの最高気温は、2020年が90度、2019年が90度、2018年が90度、2017年が79度、2016年が71度でした。過去5年間のワシントンDCの7月4日の平均気温は何度ですか？
答え：
84
回答：
平均気温を求めるには、すべての気温を合計し、それを年数で割ります。

気温は次の通りです：90（2020年）、90（2019年）、90（2018年）、79（2017年）、71（2016年

まず、気温を合計します。
90 + 90 + 90 + 79 + 71 = 420

次に、その合計を年数で割ります。
420 ÷ 5 = 84

過去5年間のワシントンDCの7月4日の平均気温は84度です。
抜粋：
平均気温を求めるには、すべての気温を合計し、それを年数で割ります。

気温は次の通りです：90（2020年）、90（2019年）、90（2018年）、79（2017年）、71（2016年

まず、気温を合計します。
90 + 90 + 90 + 79 + 71 = 420

次に、その合計を年数で割ります。
420 ÷ 5 = 84

過去5年間のワシントンDCの7月4日の平均気温は84度です。
-------------------- 問題：
ルネは60分間で30ページを読み終えることができます。ルルは60分間で27ページ、チェリーは60分間で25ページを読むことができます。彼らが240分間読書をした場合、合計で何ページ読み終えることができるでしょうか？
答え：
328
回答：
2人が合計で何ページ読んだかを求めるには、240分間にそれぞれが何ページ読めるかを計算し、その数字を合計する必要があります。

ルネは60分間に30ページ読めます。ルネが240分間に何ページ読めるかを求めるには、240を60で割って、その結果に30を掛けます。

240 / 60 = 4
4 * 30 = 120

ルルは60分間に27ページ読めます。240分間にルルが読めるページ数を求めるには、240を60で割って、その結果に27を掛けます。

240 / 60 = 4
4 * 27 = 108

チェリーは60分間で25ページ読めます。240分間でチェリーが何ページ読めるかを知るには、240を60で割って、その結果に25を掛けます。

240 / 60 = 4
4 *
抽出：
彼らが読み終えたページの総数を求めるには、240分間に各人が何ページ読めるかを計算し、それらの数字を合計する必要があります。

ルネは60分間に30ページ読み終えることができます。240分間にルネが何ページ読めるかを求めるには、240を60で割って、その結果に30を掛けます。

240 / 60 = 4
4 * 30 = 120

ルルは60分間で27ページを読むことができます。ルルが240分間で何ページ読めるかを知るには、240を60で割って、その結果に27を掛けます。

240 / 60 = 4
4 * 27 = 108

チェリーは60分間で25ページを読むことができます。チェリーが240分間で何ページ読めるかを知るには、240を60で割って、その結果に25を掛けます。

240 / 60 = 4
4 *
-------------------- 問題：
マーティンが小さい方の鐘を鳴らす回数は、大きい方の鐘を鳴らす回数の4倍よりも1/3倍多い。 マーティンが両方の鐘を合わせて52回鳴らす場合、大きい方の鐘を鳴らす回数は何回か？
答え：
36
回答：
マーティンが大きい方の鐘を鳴らす回数を「B」、小さい方の鐘を鳴らす回数を「S」と定義する。

問題によると、マーティンが小さい方の鐘を鳴らす回数は、大きい方の鐘を鳴らす回数の3分の1よりも4回多い。したがって、次のような方程式が成り立つ。

S = (1/3)B + 4

また、マーティンが両方の鐘を鳴らす回数の合計は52回である。したがって、方程式は次のようになる。

B + S = 52

この方程式を解くには、最初の式で求めたSの値を2番目の式に代入します。

B + ((1/3)B + 4) = 52

同類項をまとめる：

B + (1/3)B + 4 = 52

分数を消すために、すべての項に3を掛ける：

3B + B + 12 = 156

同類項をまとめる：

4B + 12 = 156


抽出された
：
マーティンが大きな鐘を鳴らす回数を「B」、小さな鐘を鳴らす回数を「S」と定義しよう。

問題によると、マーティンは小さな鐘を大きな鐘の1/3よりも4倍多く鳴らす。したがって、方程式は次のようになる。

S = (1/3)B + 4

マーティンが両方の鐘を鳴らす回数の合計は52回であることもわかっています。したがって、方程式は次のようになります。

B + S = 52

この方程式の組を解くには、最初の式で求めたSの値を2番目の式に代入します。

B + ((1/3)B + 4) = 52

同類項をまとめる：

B + (1/3)B + 4 = 52

分数をなくすために、すべての項に3を掛ける：

3B + B + 12 = 156

同類項をまとめる：

4B + 12 = 156

Sub
--------------------
質問： バートは毎日新聞のクロスワードパズルを解いています。 彼は2週間に1本のペースで鉛筆を使い切っています。 平均すると、鉛筆1本を使い切るのに1050語のパズルを解く必要があります。 クロスワードパズル1つには平均何語の単語が含まれているでしょうか？
答え：
75
回答：
各クロスワードパズルの平均語数を求めるには、使用された総語数（鉛筆の使用数に相当）をその期間に解いたパズルの数で割る必要があります。

鉛筆は2週間もちます。1週間は7日間なので、バートは1本の鉛筆で2×7=14個のパズルを解くことができます。

2週間から2週間と1日までの日数は42.5日（または43日）で、1日は43日または2週間です。この問題の答えをできるだけ簡単にすると、14日とします。

バートは2週間ごとに1本の鉛筆を使い切ってパズルを解くので、14日間で1本の鉛筆を使い切ります。1050語が1本の鉛筆に相当するとすると、1050を14で割る必要があります。

1050 / 14 = 75

抽出：
各クロスワードパズルの平均語数を求めるには、使用された語数の合計（鉛筆の使用数であり、それを単語数に換算した数）を、その期間に解かれたパズルの数で割る必要があります。

鉛筆は2週間もちます。1週間は7日間なので、バートは1本の鉛筆で2×7=14個のクロスワードパズルを解くことができます。

2週間と1日は43日間、つまり2週間です。この問題の答えをできるだけ簡単にすると、14とします。

バートは2週間ごとにこのパズルを解くために鉛筆を1本使い切るので、14日間で1本の鉛筆を使い切ります。1050語が1本の鉛筆を使うとすると、1050を14で割る必要があります。

1050÷14=75
-------------------- 問題：
マットは2ポンドの小麦粉で1ダース分のクッキーを一度に作ることができます。彼は1袋5ポンドの小麦粉を4袋使います。ジムが15枚のクッキーを食べた場合、残りのクッキーは何枚ですか？
答え：
105
回答：
残りのクッキーの枚数を求めるには、まずマットが合計で何枚のクッキーを作れるかを知る必要があります。

マットは2ポンドの小麦粉を使って1ダースのクッキーを焼きます。4袋（合計20ポンド）の小麦粉で何バッチ作れるかを知るには、小麦粉の総量をバッチごとの小麦粉使用量で割る必要があります。

20ポンド÷2ポンド/バッチ＝10バッチ

マットは1バッチにつき1ダース（12個）のクッキーを作るので、バッチ数と1バッチあたりのクッキー数を掛けることができます。

10バッチ×1バッチあたり12個のクッキー＝120個のクッキー

次に、ジムが食べたクッキーの数（15個）をクッキーの総数から差し引きます。

120個のクッキー - 15個のクッキー = 105個のクッキー

残っているクッキーは105個です。
抽出：
残っているクッキーの数を調べるには、まずマットが合計で何個のクッキーを作れるかを知る必要があります。

マットは2ポンドの小麦粉を使って1ダースのクッキーを1バッチ作ります。4袋（合計20ポンド）の小麦粉で何バッチ作れるかを調べるには、小麦粉の総量を使用するバッチごとの小麦粉の量で割る必要があります。

20ポンド÷2ポンド/バッチ＝10バッチ

マットは1バッチにつき1ダース（12個）のクッキーを作るので、バッチ数に1バッチあたりのクッキーの数を掛けることができます。

10バッチ×1バッチあたり12個のクッキー＝120個のクッキー

次に、ジムが食べたクッキーの数（15個）をクッキーの総数から差し引きます。

120個のクッキー - 15個のクッキー = 105個のクッキー

残りのクッキーは105個です。
-------------------- 問題：
ジェームズは1週間で500個の缶を集め、ブリキの家を建てることにしました。1日目に彼は50個の缶を集めました。2日目には、その3倍の数を集めることができました。3日目には、2日目に集めた数の50缶少ない缶を集めた。残りの日数で同じ数の缶を集めた場合、残りの週の各日の缶の数はいくつになるか？
答え：
50
回答：
まず、ジェームズが最初の3日間で集めた缶の総数を計算する必要がある。

初日に彼は50個の缶を集めました。
2日目にはその3倍の3×50=150個の缶を集めました。
3日目には、2日目に集めた数より50個少ない150-50=100個の缶を集めました。

最初の3日間で集めた缶の合計数 = 50 + 150 + 100 = 300 缶。

ジェームズは合計で500個の缶を集める必要があり、最初の3日間で300個の缶を集めたので、残りの週で500 - 300 = 200個の缶を集める必要があります。

今週はあと4日残っているので、1日に集める缶の数を求めるには、残りの缶の数を割ります。
抽出：
まず、ジェームズが最初の3日間で集めた缶の総数を計算する必要があります。

1日目に彼は50個の缶を集めました。
2日目は、その3倍の3×50=150個を集めました。
3日目は、2日目に集めた数より50個少ないので、150-50=100個を集めました。

最初の3日間で集めた缶の合計数 = 50 + 150 + 100 = 300 缶。

ジェームズは合計で500個の缶を集める必要があり、最初の3日間で300個の缶を集めたので、残りの週で500 - 300 = 200個の缶を集める必要があります。

今週はあと4日あるので、1日に集める缶の数を求めるには、残りの缶の数を
--------------------で割ります。
問題： ジェリービーンズの瓶には、青いジェリービーンズが14個、紫のジェリービーンズが26個、オレンジのジェリービーンズが40個入っています。 瓶の中にジェリービーンズが200個ある場合、赤い色のジェリービーンズは何個あるでしょうか？
答え：
120個
回答：
情報を分析してみましょう。
瓶の中には合計200個のジェリービーンズが入っています。青、紫、オレンジのジェリービーンズの数は分かっています。赤いジェリービーンズの数を求めるには、まず青、紫、オレンジのジェリービーンズの数を合計し、その合計から瓶の中のジェリービーンズの合計を引きます。

青、紫、オレンジのジェリービーンズの合計は、14 + 26 + 40 = 80 です。

さて、赤いジェリービーンズの数を求めるには、合計数からこの合計を引きます。200 - 80 = 120

しかし、問題文には赤いジェリービーンズについて一切触れられておらず、青、紫、オレンジのジェリービーンズについてのみ言及されています。
抜粋：
情報を分析してみましょう。
瓶の中のゼリービーンズの総数は200個です。青、紫、オレンジのゼリービーンズの数は分かっています。赤いゼリービーンズの数を求めるには、まず青、紫、オレンジのゼリービーンズの数を合計し、次にその合計数から瓶の中のゼリービーンズの総数を引きます。

青、紫、オレンジのジェリービーンズの合計は、14 + 26 + 40 = 80 です。

赤いジェリービーンズの数を求めるには、合計数からこの合計を引きます。200 - 80 = 120

しかし、問題文には赤いジェリービーンズについて一切触れられておらず、青、紫、オレンジのジェリービーンズについてのみ言及されています。
-------------------- 問題：
大人5人と子供2人が映画を見に行き、12ドル分の軽食を購入した。この旅行の合計費用は76ドルだった。子供1人分のチケットが7ドルだとすると、大人1人分のチケットはドルでいくらになるだろうか？
答え：
10
回答：
大人1人分のチケットの価格を「A」と表記しよう。
大人5人、子供2人なので、大人チケットの合計金額は5A、子供チケットの合計金額は2×7=14ドルです。
合計金額は、5A+14+12=76と表すことができます。

両辺から26（14＋12）を引くと、
5A＝50

両辺を5で割ると、
A＝10

大人用チケットの価格は10ドルとなります。
抜粋：
大人用チケットの価格を「A」と表記します。
大人5人、子供2人なので、大人チケットの合計金額は5A、子供チケットの合計金額は2×7=14ドルです。
合計金額は、5A+14+12=76と表すことができます。

両辺から26（14+12）を引くと、
5A=50

両辺を5で割ると、
A = 10

となります。 大人のチケットの価格は10ドルです。
--------------------
問題：ジャネルは緑色のビー玉を26個持っていました。その後、青色のビー玉を6袋買いました。それぞれの袋には10個のビー玉が入っていました。 彼女は緑のビー玉6個と青のビー玉8個を贈り物として用意し、友人にそれをあげました。 ジャネルは今、何個のビー玉を持っているでしょうか？
答え：
72
回答：
ジャネルが今、何個のビー玉を持っているかを知るには、まず彼女が青のビー玉を何個買ったかを知り、それから彼女が贈り物としてあげたビー玉の数を、彼女が持っているビー玉の総数から差し引く必要があります。

まず、ジャネルが購入した青いビー玉の数を求めます。
6袋*1袋あたり10個のビー玉=60個の青いビー玉

ジャネルはもともと26個の緑のビー玉を持っており、6個の緑のビー玉を贈り物としてあげたので、彼女の手元には
26-6=20個の緑のビー玉が残ります。

ジャネルは6個の緑のビー玉と8個の青いビー玉を贈り物としてあげたので、

したがって、ジャネルが贈り物を手放す前に持っていたビー玉の合計数は、
緑のビー玉26個＋青のビー玉60個＝86個

贈り物を手放した後、ジャネルが持っているのは、
合計86個のビー玉から贈り物として手放した14個を引いた72個

<
抜粋：
ジャネルが現在持っているビー玉の数を調べるには、まず彼女が購入した青いビー玉の数を調べ、次に彼女がプレゼントしたビー玉の数を彼女の持っているビー玉の総数から差し引く必要があります。

まず、ジャネルが購入した青いビー玉の数を調べます。
6袋*1袋あたり10個のビー玉=60個の青いビー玉

ジャネルはもともと緑色のビー玉を26個持っていて、そのうち6個をプレゼントしたので、彼女の手元には
26 - 6 = 20個の緑色のビー玉が残っている

ジャネルは緑色のビー玉を6個 + 青色のビー玉を8個 = 合計14個のビー玉をプレゼントした

したがって、プレゼントをあげる前にジャネルが持っていたビー玉の合計数は、
緑のビー玉26個 + 青のビー玉60個 = 86個

プレゼントをあげた後、ジャネルの手元には
合計86個のビー玉 - プレゼントした14個のビー玉 = 72個のビー玉

<
--------------------
問題：
シエナは1週間の間に、ブラウザで調べた研究から30のウェブサイトページをブックマークする。現在、シエナのブックマークライブラリには400ページがブックマークされています。3月末には、ブックマークライブラリには何ページあるでしょうか？
回答：
1330ページ
回答：
シエナは毎日30ページをブックマークし、1ヶ月は31日あるので、1ヶ月にブックマークしたページの合計数を計算する必要があります。

30ページ/日×31日＝930ページ

しかし、すでにブックマークされているページも考慮する必要があります。

930ページ（1か月間の新規ブックマーク）＋400ページ（すでにブックマークされているページ）＝1330ページ

したがって、3月末には、シエナのブックマークライブラリには1330ページが登録されていることになります。
抜粋：
シエナが毎日30のウェブサイトページをブックマークし、1ヶ月は31日あるので、1ヶ月にブックマークしたページの総数を計算する必要があります。

30ページ/日*31日=930ページ

しかし、すでにブックマークされているページも考慮する必要があります。

930ページ（1か月間の新規ブックマーク）＋400ページ（すでにブックマークされているページ）＝1330ページ

したがって、3月末には、シエナのブックマークライブラリには1330ページが登録されていることになります。
-------------------- 問題：
ある年老いた海賊が、自分の宝を島に残したいと考えています。海賊は3500枚の金貨を持っています。海賊はこれを5つの宝箱に均等に分配したいと考えています。さらに、海賊は合計500枚の銀貨と、銀貨の2倍の銅貨を、すべて均等に分配して宝箱に入れます。それぞれの宝箱には合計何枚のコインが入るでしょうか？
回答：
1000枚
回答：
各箱の硬貨の枚数を求めるには、それぞれの硬貨の総数を計算し、それを箱の数で割る必要があります。

金貨の総数は3500枚で、海賊はそれを5つの箱に均等に分配したいと考えています。

3500÷5=1箱あたり700枚の金貨。

海賊はさらに、500枚の銀貨を5つの宝箱に均等に分配します。
500÷5=100枚の銀貨が1つの宝箱に入ります。

海賊は、銀貨の2倍の青銅貨を分配します。つまり、2×500=1000枚の青銅貨が1つの宝箱に入ります。そして、海賊はこれを5つの宝箱に均等に分配します。
1000 ÷ 5 = 1つの宝箱につき200枚の青銅貨。

次に、それぞれの宝箱のすべてのコインを合計します。
700枚（金貨）＋100枚（銀貨）＋200枚（青銅貨）＝1つの宝箱につき1000枚のコイン。

答えは、1つの宝箱につき1000枚のコインです。
抽出：
それぞれの宝箱に入っているコインの枚数を求めるには、各コインの総数を計算し、それを宝箱の数で割る必要があります。

金貨の総数は3500枚で、海賊はそれを5つの宝箱に均等に分配したいと考えています。

3500÷5=700枚の金貨が1つの宝箱に入ります。

海賊はさらに、500枚の銀貨を5つの宝箱に均等に分配します。
500÷5=100枚の銀貨が1つの宝箱に入ります。

海賊は、銀貨の2倍の青銅貨を分配します。つまり、2×500=1000枚の青銅貨が1つの宝箱に入ります。そして、海賊はこれを5つの宝箱に均等に分配します。
1000÷5=1つの箱につき200枚の青銅貨。

次に、それぞれの箱の貨幣をすべて合計します。
700（金貨）＋100（銀貨）＋200（青銅貨）＝1つの箱につき1000枚の貨幣。

答えは、1つの箱につき1000枚の貨幣
-------------------- 問題：
エフライムは自分の店で販売するネックレスを作る機械を2台持っています。日曜日に最初の機械は45個のネックレスを作りました。2番目の機械は最初の機械の2.4倍の数のネックレスを作りました。日曜日に合計で何個のネックレスが作られたでしょうか？
答え：
153
回答：
推論＞
日曜日に作られたネックレスの総数を求めるには、まず第2の機械で作成されたネックレスの数を計算する必要があります。第2の機械は第1の機械の2.4倍の数のネックレスを作ったので、第1の機械で作成されたネックレスの数（45）に2.4を掛けます。

次に、第1と第2の機械で作成されたネックレスの数を合計します。

</reasoning>

<answer>
第1の機械は45個のネックレスを製造し、第2の機械は45×2.4=108個のネックレスを製造した。したがって、日曜日に製造されたネックレスの合計数は45+108=153個である。
抽出：
最初の機械で45個のネックレスが作られ、2番目の機械で45×2.4=108個のネックレスが作られた。したがって、日曜日に作られたネックレスの合計数は45+108=153個である。
-------------------- 問題：
ザックはクッキーを焼いて販売しました。月曜日に彼は32個のクッキーを焼きました。火曜日に砂糖が切れてしまい、前日に焼いた数の半分しか焼けませんでした。水曜日に彼は火曜日に焼いた数の3倍の数のクッキーを焼きましたが、そのうち4個は彼の兄弟が食べてしまいました。3日間の終わりに彼の手元には何個のクッキーが残っていたでしょうか？
答え：
92
回答：
1日ずつ見てみましょう。

月曜日、ザックは32個のクッキーを焼きました。

火曜日、砂糖が切れてしまい、前日の半分しか焼けず、32/2=16個のクッキーを焼きました。

水曜日に、彼は火曜日に焼いた数の3倍のクッキーを焼きました。しかし、彼の弟はそのうちの4個を食べました。そのため、ザックが残したクッキーは48 - 4 = 44個でした。

3日間の終わりに焼いたクッキーの総数を求めるには、各日に焼いたクッキーの数を足します。

32（月曜日）＋16（火曜日）＋44（水曜日）＝92個のクッキー

<answer>92</answer>
抽出：
92
--------------------
マークとジェームズは、ゲームをするために12面体のサイコロを合計14個必要としています。マークは10個のサイコロの袋を持っており、そのうち60%が12面体です。 ジェームズは8個のサイコロを持っており、そのうち75%が12面体です。 ゲームをするために、少年たちは何個のサイコロを買わなければならないでしょうか？
答え：
2
回答：
# まず、マークのサイコロバッグに入っている12面体のサイコロの数を調べます。マークは10個のサイコロを持っており、そのうち60%が12面体です。
10個の60%は、10 * 0.6 = 6で計算できる

マークは12面体のサイコロを6個持っている。

次に、ジェームズの袋の中の12面体のサイコロの数を求めよう。ジェームズはサイコロを8個持っており、そのうち75%が12面体である。
8個の75%は、8×0.75=6で計算できます。

ジェームズは12面体のサイコロを6個持っています。

マークとジェームズが一緒に持っている12面体のサイコロは6+6=12個なので、合計14個のサイコロが必要なので、14-12=2個のサイコロをまだ購入する必要があります。

答え：2個のサイコロを買う必要があります。
回答：
まず、エリンがそれぞれの場所で見たトロルの数を把握しましょう。

1. エリンは小道のそばに6匹のトロルを見ました。
2. 小道のそばにいるトロルの4倍の数が橋の下にいて、その数は6×4=24です。
   橋の下にいるトロルの数は、それより6匹少ないので、24-6=18匹です。
3. 平野にいるトロールの数は橋の下にいるトロールの数の半分なので、平野にいるトロールの数は18÷2=9です。

次に、エリンが見たトロールの数をすべて合計してみましょう。
6（小道沿い）＋18（橋の下）＋9（平野）＝33

答え：33



抽出：
33
-------------------- 問題：
カレンは娘のランチにピーナッツバターサンドイッチを詰めるのは週の2日でランダムに選ばれた曜日である。他の3日の学校では、ハムサンドイッチを詰める。彼女はランダムに選ばれた1日にケーキを詰め、他の4日間はクッキーを詰める。カレンがハムサンドイッチとケーキを同じ日に詰める確率は、パーセントで表すと何パーセントか？
回答：
12
回答：
1週間のうち学校がある日は5日ある。任意の日にハムサンドイッチを選ぶ確率は3/5である。なぜなら、5日間のうち3日間がハムサンドイッチだからである。任意の日にケーキを詰める確率は1/5である。なぜなら、5日間のうち1日がケーキだからである。

ハムサンドイッチとケーキの両方を詰める確率を求めるには、それぞれの確率を掛ければよい。ただし、これらは独立した2つの事象であるため、日々の重複を考慮する必要はない。

ハムサンドイッチを詰める日がハムサンドイッチの日でもある確率は、(3/5) * (1/5) = 3/25 です。

この確率をパーセントで表すには、3 を 25 で割って 100 を掛けます。

(3/25) * 100 = 12

したがって、カレンがハムサンドイッチとケーキを同じ日に詰める確率は12%となります。
抜粋：
1週間のうち学校がある日は5日です。任意の日にハムサンドイッチを選ぶ確率は3/5です。なぜなら、5日間のうち3日間はハムサンドイッチだからです。任意の日にケーキを詰める確率は1/5です。なぜなら、5日間のうち1日はケーキだからです。

ハムサンドイッチとケーキの両方を詰めるという2つの事象が同じ日に起こる確率を求めるには、それぞれの確率を掛け合わせます。しかし、これらは独立した2つの事象であるため、日々の重複を考慮する必要はありません。

ハムサンドイッチを詰める日がケーキの日でもある確率は、(3/5) * (1/5) = 3/25 となります。

この確率をパーセントで表すには、3を25で割って100を掛けます。

(3/25) * 100 = 12%

したがって、カレンがハムサンドイッチとケーキを同じ日に詰める確率は12
です。 --------------------- 問題：
4層のピラミッドが作られており、各層にはその上の層よりも3倍の砂岩ブロックが置かれている。最上層にはブロックが1つ置かれている。ピラミッドには何個の砂岩ブロックがあるか？
答え：
40
回答：
ブロックの総数を求めるには、各層に置かれているブロックの数を計算してみましょう。

1. 最上層にはブロックが1つ置かれている。
2. 2番目の層には、1番目の層の3倍の数のブロックがあり、1×3=3個のブロックです。
3. 3番目の層には、2番目の層の3倍の数のブロックがあり、3×3=9個のブロックです。
4. 4番目の層には、3番目の層の3倍の数のブロックがあり、9×3=27個のブロックです。

それでは、各層にあるブロックの数を合計して、ブロックの総数を求めましょう。
1 + 3 + 9 + 27 = 40

答え：ピラミッド内の砂岩ブロックの総数は40です。



抽出：
ピラミッド内の砂岩ブロックの総数は40です。
-------------------- 問題：
ジョーの新しい空調システムの費用は2万ドルです。このシステムには2つの調整ゾーンがあり、それぞれに5つの通気口があります。通気口1つあたりのシステムの費用はドルでいくらでしょうか？
回答:
2,000
回答:
通気口1つあたりの費用を求めるには、システムの総費用を通気口の総数で割る必要があります。

ゾーンは2つあり、それぞれに5つの通気口があります。したがって、通気口の総数は2×5=10個です。

通気口1つあたりの費用 = 総費用 / 通気口の総数
= 20,000ドル / 10個の通気口
= 1個あたり2,000ドル

<answer>2,000ドル</answer>
抽出:
2,000ドル
-------------------- 問題:
トレントは家からバス停まで4ブロック歩いた。それからバスで7ブロック移動して図書館へ行った。その後、同じ道を帰ってきた。トレントは合計で何ブロック移動したか？
答え：
22
回答：
トレントが移動したブロックの合計数を求めるには、まず片道の距離を割り出し、次に図書館まで行ってから帰ってきたので、その数字を2倍する。

彼の家から図書館までの片道の距離は、徒歩4ブロック＋バス7ブロック＝11ブロックです。

同じ道順で家に戻ったので、さらに11ブロック移動したことになります。

したがって、トレントが移動したブロックの合計数は、11＋11＝22ブロックとなります。
抜粋：
トレントが移動したブロックの合計数を求めるには、まず片道の距離を割り出し、次に図書館に行ってから家に戻ってきたので、その数字を2倍します。

家から図書館までの片道の距離は、徒歩で4ブロック、バスで7ブロック、合計11ブロックです。

家に戻る時も同じ道を通ったので、さらに11ブロック移動したことになります。

したがって、トレントが移動したブロックの合計は11＋11＝22ブロックです。
--------------------
質問：リーは10ドル、友人は8ドル持っていました。彼らはレストランに行き、6ドルのチキンウィングと4ドルのチキンサラダを注文しました。さらに、1ドルのソーダを2本頼みました。税金は3ドルでした。合計でいくらのおつりをもらうべきでしょうか？
回答：
3
回答：
食べ物と飲み物の合計金額を計算してみましょう。

食べ物と飲み物の合計金額は、6ドル（手羽先）＋4ドル（チキンサラダ）＋2ドル（ソーダ2本）＝12ドルです。
税金は3ドルです。
したがって合計金額は12ドル＋3ドル＝15ドルとなります。

次に、彼らが持っていた合計金額、10ドル＋8ドル＝18ドルを足します。
彼らが受け取るべきおつりの合計は、18ドル（合計金額）－15ドル（合計金額）＝3ドルです。

（答え）
3ドル
抽出：
食べ物と飲み物の合計金額を計算してみましょう。

料理と飲み物の合計金額は、手羽先6ドル、チキンサラダ4ドル、ソーダ2本2ドルで、合計12ドルです。
税金は3ドルです。
合計金額は12ドル＋3ドルで15ドルです。

次に、合計金額10ドル＋8ドルで18ドルを足します。
彼らが受け取るべきおつりの合計は、18ドル（合計金額）から15ドル（合計金額）を引いた3ドルです。

（答え）
3ドル
--------------------
問題：
アンディは家から学校まで50メートル歩きます。学校が終わると、家に戻ってから市場に行きます。合計で140メートル歩くとすると、家から市場までの距離は何メートルですか？
答え：
40
回答：
## ステップ 1: アンディが1日に歩く合計距離を求める
アンディが1日に歩く合計距離は140メートルと与えられています。

## ステップ 2: 家から学校、学校から家までの距離を計算する
アンディは家から学校まで50メートル歩き、その後、また50メートル歩いて家に戻ります。つまり、家から学校までの合計距離は50＋50＝100メートルです。

ステップ3：家から学校までの距離を差し引いた残りの距離を計算する
アンディが合計140メートルを歩き、家から学校までの距離が100メートルなので、家から市場までの距離を算出するには、合計距離から家から学校までの距離を差し引く必要があります。 残りの距離 = 合計距離 - 家から学校までの距離 = 140 - 100 = 40メートル。


抜粋：
## ステップ 1: アンディが1日に歩く合計距離を求める
アンディが1日に歩く合計距離は140メートルとされています。

## ステップ 2: 家から学校までの距離と学校から家までの距離を計算する
アンディは家から学校まで50メートル歩き、その後、また50メートル歩いて家に戻ります。つまり、家から学校までの合計距離は50＋50＝100メートルです。

ステップ3：家から学校までの距離を差し引いた残りの距離を計算する
アンディが合計140メートルを移動し、家から学校までの距離が100メートルであるため、家から市場までの距離を算出するには、合計距離から家から学校までの距離を差し引く必要があります。 残りの距離 = 合計距離 - 家から学校までの距離 = 140 - 100 = 40メートル
-------------------- 問題：
タラは1000ドルのノートパソコンの購入を計画しています。あるコンピューターショップでは、頭金として20%を支払えば、月々65ドルの分割払いが可能とのことです。タラが頭金としてさらに20ドルを支払う場合、4か月間支払った後の残金はいくらになるでしょうか？
回答：
520
回答：
## 最初の計算：分割払い総額と頭金。
ノートパソコンの価格は1000ドルで、毎月の分割払いは65ドルです。20%の頭金1000ドルを支払った後、タラは20/100*1000=200ドルを支払っています。したがって、頭金20ドルを追加する必要があり、合計頭金は220ドルとなります。

## 頭金支払い後の残金
残りの金額は、1000 - 220 = 780ドルとなります。

## 4か月間の毎月の支払い
4か月間、毎月65ドルを支払った場合、支払総額は4 * 65 = 260ドルとなります。

## 4か月後の残高
残金は、頭金から4か月分の支払額を差し引いた残額となります。つまり、780 - 260 = 520ドルとなります。

最終回答：
<reasoning>頭金と月々の分割支払額を考慮すると、
以下が導かれます。
## 最初の計算：分割支払総額と頭金。
ノートパソコンの価格は1000ドルで、毎月の分割払いは65ドルです。頭金20%の1000ドルを支払った後、タラは20/100 * 1000 = 200ドルを支払っています。 したがって、頭金は220ドルとなります。

## 頭金支払い後の残額
残りの金額は、1000 - 220 = 780ドルとなります。

## 4か月間の毎月の支払い
4か月間、毎月65ドルを支払った場合、支払総額は4 * 65 = 260ドルとなります。

## 4か月後の残高
残金は、頭金から4か月分の支払額を差し引いた残額となります。つまり、780 - 260 = 520ドルとなります。

最終回答：
<reasoning>頭金と月々の分割支払額を考慮すると、

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [11]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:27<00:00, 27.70s/it, est. speed input: 1.41 toks/s, output: 17.12 toks/s]


'**Calculating Pi**\n\nPi (π) is a mathematical constant representing the ratio of a circle\'s circumference to its diameter. It\'s approximately equal to 3.14159, but it\'s an irrational number, so its decimal representation goes on infinitely.\n\n**Leibniz Formula**\n\nOne way to calculate pi is using the Leibniz formula:\n\nπ/4 = 1 - 1/3 + 1/5 - 1/7 + 1/9 - ...\n\nThis formula is a series that converges to π/4. We can use this formula to calculate pi by summing the terms until we reach a desired level of precision.\n\n**Example Code**\n\nHere\'s a Python function to calculate pi using the Leibniz formula:\n\n```python\ndef calculate_pi(n_terms):\n    pi = 0.0\n    for i in range(n_terms):\n        sign = (-1) ** i\n        pi += sign * (1 / (2 * i + 1))\n    pi *= 4  # Multiply by 4 to get the actual value of pi\n    return pi\n\n# Calculate pi with 1000 terms\npi = calculate_pi(1000)\nprint(f"Approximate value of pi: {pi:.10f}")\n```\n\n**BBP Formula**\n\nAnother way to calculate p

And now with the LoRA we just trained with GRPO - we first save the LoRA first!


In [12]:
model.save_lora("/content/drive/MyDrive/Colab Notebooks/weights/Llama3.1_8B-GRPO/grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("/content/drive/MyDrive/Colab Notebooks/weights/Llama3.1_8B-GRPO/grpo_saved_lora"),
)[0].outputs[0].text

output

これは難しい課題です。なぜなら、円周率は正確に計算できない非合理的な数だからです。しかし、現在知られている最も効率的な方法であるガウス・ルジャンドル法を使用して近似値を提供することができます。AGMは次のように定義されます。 AGM(a, b) = sqrt((a + b)^2 / 2) ここで、`a`と`b`は2つの数値です。 アルゴリズムは、以下の式にAGMを繰り返し適用することで動作します。 以下は、ガウス・レグレアル法のシンプルなPythonによる実装です。 a_new = (a + b) / 2.0b_new = sqrt(a * b)t_new = t - p * pow((a - a_new), 2)p += 1.0a = a_newb = b_newt = t_new  

私たちの推論モデルは優れています。ただし、1時間ほどしか訓練していないので、常に正しいわけではありません。
シーケンスの長さを長くして、より長い期間訓練すれば、さらに良くなるでしょう！

<a name="Save"></a>
### Saving to float16 for VLLM

float16への直接保存もサポートしています。float16の場合はmerged_16bit、int4の場合はmerged_4bitを選択してください。また、loraアダプターをフォールバックとして使用することもできます。Hugging Faceアカウントにアップロードするには、push_to_hub_mergedを使用してください！https://huggingface.co/settings/tokensにアクセスすると、個人用トークンを取得できます。

In [14]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
GGUF / llama.cpp に保存するには、現在ネイティブでサポートされています。llama.cpp をクローンし、デフォルトで q8_0 に保存します。q4_k_m のようなすべてのメソッドが使えます。ローカル保存には save_pretrained_gguf を使用し、HF へのアップロードには push_to_hub_gguf を使用します。

サポートされている一部の量子化メソッド（全リストは Wiki ページを参照）：

q8_0 - 高速変換。リソースの使用量は多いが、概ね許容できる。
q4_k_m - 推奨。attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用し、残りにQ4_Kを使用する。
q5_k_m - 推奨。attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用し、残りにQ5_Kを使用する。
[NEW] Ollamaへの自動エクスポートと微調整を行うには、Ollamaノートブックをお試しください。

GGUF / llama.cpp に保存するには、現在ネイティブでサポートされています。llama.cpp をクローンし、デフォルトで q8_0 に保存します。q4_k_m のようなすべてのメソッドを許可します。ローカル保存には save_pretrained_gguf を使用し、HF へのアップロードには push_to_hub_gguf を使用します。

サポートされている一部の量子化メソッド（全リストは Wiki ページを参照）：

q8_0 - 高速変換。リソースの使用量は多いが、概ね許容できる。
q4_k_m - 推奨。attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用し、残りにQ4_Kを使用する。
q5_k_m - 推奨。attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用し、残りにQ5_Kを使用する。
[NEW] Ollamaへの自動エクスポートと微調整を行うには、Ollamaノートブックをお試しください。

In [15]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

次に、`model-unsloth.gguf`ファイルまたは`model-unsloth-Q4_K_M.gguf`ファイルをllama.cppまたはJanやOpen WebUIのようなUIベースのシステムで使用します。Janは[こちら](https://github.com/janhq/jan)から、Open WebUIは[こちら](https://github.com/open-webui/open-webui)からインストールできます

インストールが完了しました！Unslothについて何か質問があれば、[Discord](https://discord.gg/unsloth)チャンネルをご利用ください！バグを見つけた場合や、LLMの最新情報を入手したい場合、またはヘルプが必要な場合、プロジェクトに参加したい場合など、Discordにぜひご参加ください！

その他のリンク：
1. Llama 3.2 会話型ノートブック。 [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Ollamaへの微調整の保存。[フリーノートブック](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 ビジョン微調整 - 放射線画像のユースケース。 [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. DPO、ORPO、継続的な事前トレーニング、会話の微調整などについては、[ドキュメント](https://docs.unsloth.ai/get-started/unsloth-notebooks)のノートブックをご覧ください！

<div class=「align-center」>
  <a href=「https://unsloth.ai」><img src=「https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png」 width=「115」></a>
  <a href=「https://discord.gg/unsloth」><img src=「https://github.com/unslothai/unsloth/raw/main/images/Discord.png」 width=「145」></a>
  <a href=「https://docs.unsloth.ai/」><img src=「https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true」 width=「125」></a>

  ヘルプが必要な場合はDiscordに参加してください + ⭐️ <i>Githubでスターを付ける</i> ⭐️
</div>
